# ML_25M Classic ML

In [ ]:
import os
import time
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

seed = 42
np.random.seed(seed)
random.seed(seed)
df = pd.read_csv('dataset.csv')


## Data Preprocessing

In [ ]:
X = df.drop('rating', axis=1)
Y = df['rating']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Scree plot
pca = PCA(.95)
pca.fit(X_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

print(pca.n_components_)

In [ ]:
# Kaiser criterion
pca = PCA()
pca.fit(X_train)
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('explained variance')
plt.show()

print(pca.n_components_)

# PCA
pca = PCA(n_components=0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
# PCA
pca = PCA(n_components=556)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:

print(f'Number of training samples: {X_train.shape}')
print(f'Number of testing samples: {X_test.shape}')
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'R2'])


# Models

## Support Vector Regression

In [ ]:
# SVR with Bayesian Optimization
def svr_cv(C, gamma, epsilon):
    val = cross_val_score(SVR(C=C, gamma=gamma, epsilon=epsilon), X_train, Y_train, scoring="neg_mean_squared_error", cv=5)
    return np.mean(val)

# Bounded region of parameter space
pbounds = {'C': (0.1, 100), 'gamma': (0.0001, 0.1), 'epsilon': (0.1, 1)}

optimizer = BayesianOptimization(
    f=svr_cv,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=2,
    n_iter=10,
)   

print(optimizer.max{'params'})

# SVR
svr = SVR(C=optimizer.max['params']['C'], gamma=optimizer.max['params']['gamma'], epsilon=optimizer.max['params']['epsilon'])
svr.fit(X_train, Y_train)
Y_pred = svr.predict(X_test)

# Evaluate the model
mse = np.mean((Y_pred - Y_test)**2)
rmse = np.sqrt(mse)
r2 = svr.score(X_test, Y_test)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

results = df.concat([results, pd.DataFrame({'Model': 'SVR', 'MSE': mse, 'RMSE': rmse, 'R2': r2}, index=[0])], ignore_index=True)

## Random Forest Regressor

In [ ]:
# Random Forest Regressor

best_n = 0
best_mse = float('inf')

for n in range(50, 55):
    print("Number of trees: {}".format(n), end='\r')
    rfr = RandomForestRegressor(n_estimators=n, random_state=seed)
    rfr.fit(X_train, Y_train)
    rfr_pred = rfr.predict(X_test)
    mse = np.mean((rfr_pred - Y_test) ** 2)
    if mse < best_mse:
        best_mse = mse
        best_n = n

print(f'\n\nBest number of trees: {best_n}')
print(f'Best MSE: {best_mse}')

rfr = RandomForestRegressor(n_estimators=best_n, random_state=seed)
rfr.fit(X_train, Y_train)
rfr_pred = rfr.predict(X_test)

# MSE and RMSE
mse = np.mean((rfr_pred - Y_test) ** 2)
rmse = np.sqrt(mse)
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

# R2
r2 = rfr.score(X_test, Y_test)
print(f'R2: {r2}')

results = pd.concat([results, pd.DataFrame({'Model': 'Random Forest Regressor', 'MSE': mse, 'RMSE': rmse, 'R2': r2}, index=[0])], ignore_index=True)
print(results)

## Linear Regressor

In [ ]:
#linear regression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
Y_pred = regressor.predict(X_test)

In [ ]:
# Evaluate the model
rmse = mean_squared_error(Y_pred,Y_test)

# Accuracy
r2 = r2_score(Y_test, Y_pred)

print(f'MSE: {rmse}')
print(f'R2: {r2}')

## Ridge Regression

In [ ]:
# ridge regression
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, Y_train)
ridge_pred = ridge.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(ridge_pred,Y_test)

# Accuracy
r2 = r2_score(Y_test, ridge_pred)

print(f'MSE: {rmse}')
print(f'R2: {r2}')

## Lasso Regressor

In [ ]:
# lasso regression
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.00001) # higher value of alpha results in a more sparse model. 
lasso.fit(X_train, Y_train)
lasso_pred = lasso.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(lasso_pred,Y_test)

# Accuracy
r2 = r2_score(Y_test, lasso_pred)

print(f'MSE: {rmse}')
print(f'R2: {r2}')

## Elastic Net

In [ ]:
# elastic net
from sklearn.linear_model import ElasticNet
elasticnet = ElasticNet(alpha=0.00001, l1_ratio=0.5)
elasticnet.fit(X_train, Y_train)
elastic_pred = elasticnet.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(elastic_pred,Y_test)

# Accuracy
r2 = r2_score(Y_test, elastic_pred)

print(f'MSE: {rmse}')

print(f'R2: {r2}')